In [11]:
import os, shlex
from subprocess import call, check_output

# go to the root of the repository
repo_rootdir = check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii')
os.chdir(repo_rootdir)

import os
import sys
sys.path.append("utils")

import torch

from config.config_parser import read_config
from utils.helpers import *

import utils.mesh_operations
device = get_device()

from notebooks.utils_ipywidgets import *
from IPython.display import display

import ipywidgets as widgets

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
import meshio

In [12]:
LV = Mesh("template/template.vtk").extractSubpart([1,2])
faces = np.hstack([[3] + list(x) for x in LV.triangles])

2021-02-15 22:16:56,295 [ERROR] Unsupported point attribute type: point_data for file: template/template.vtk


In [54]:
pvpython = "/home/rodrigo/ProgramFiles/Paraview/ParaView-5.7.0-MPI-Linux-Python3.7-64bit/bin/pvpython"
paraview_script = "/home/rodrigo/PhD/manuscripts/First_Journal_Paper/scripts/paraview_script.py"

In [55]:
def get_nn_in_endo(epi, endo):    
    epi_nn_in_endo = {}
    for i, _ in enumerate(epi):
        distances = np.array([np.linalg.norm(x) for x in (epi[i] - endo)])
        nns_idx = distances.argmin()
        epi_nn_in_endo[i] = nns_idx
    return epi_nn_in_endo

In [56]:
def wall_thickness(mesh, epi_nn_in_endo=None):
    endo = mesh[LV.subpartID == 1]
    epi = mesh[LV.subpartID == 2]
    if epi_nn_in_endo is None:        
        epi_nn_in_endo = get_nn_in_endo(epi, endo)
        
    wall_thickness_j = []
    for j, _ in enumerate(epi):
        wall_thickness_j.append( np.linalg.norm(epi[j] - endo[epi_nn_in_endo[j]]) )
    
    wt = np.array([np.nan] * mesh.shape[0])
    wt[LV.subpartID == 2] = wall_thickness_j
    return wt

In [57]:
def create_figures(model_name):
    
    import ExperimentClass
    
    exp_dir = os.path.join("output", model_name)
    experiment = ExperimentClass.ComaExperiment(exp_dir)
    experiment.load_model()
    prealigned_meshes = experiment.load_prealigned_meshes()
    
    endo = prealigned_meshes.mean[LV.subpartID == 1]
    epi = prealigned_meshes.mean[LV.subpartID == 2]
    epi_nn_in_endo = get_nn_in_endo(epi, endo)
    
    all_pngs = []
    model_dir = os.path.join("output", model_name)
    
    for index in range(experiment.config["z"]):
        
        ei = np.zeros(experiment.config["z"])
        ei[index] = 1.0
        z_name = "z" + str(index)
        print(z_name) 
        png_files = []
        
        z_dir = os.path.join(model_dir, "synthetic_meshes", z_name)
        os.makedirs(z_dir, exist_ok=True)
        
        for factor in range(-3, 4, 1):        
            z = ei * factor
            mesh = experiment.reconstruct_from_z(torch.Tensor(z).unsqueeze(0))
            mesh = mesh[0].detach().numpy()
            mesh = prealigned_meshes.mean + prealigned_meshes.std * mesh
            # mesh_pv = pv.PolyData(mesh[0].detach().numpy(), faces)
            
            mesh_file = os.path.join(z_dir, "{}_eq_{}.vtk".format(z_name, factor))
            
            # meshio.write_points_cells(mesh_file, mesh, {'triangle': LV.triangles})
            wt = wall_thickness(mesh, epi_nn_in_endo)
            meshio.write_points_cells(mesh_file, mesh, {'triangle': LV.triangles}, point_data={"wall_thickness": wt})
            
            png_file = os.path.join(z_dir, "{}_eq_{}_wt.png".format(z_name, factor))
            command = [pvpython] + [paraview_script] + ["--input_file"] + [mesh_file] + ["--output_file"] + [png_file]
            
            if experiment.config["procrustes_scaling"]:
                command += ["--scaled"]
                       
            command += ["--opacity", "0.5"]
            # command += ["--color_scalar", "wall_thickness"]
            # command += ["--color_max", "0.01"]
                
            call(command)
            png_files.append(png_file)
            
        png_file_pooled = os.path.join("output", model_name, "synthetic_meshes", z_name, "all_values.png")
        command = ["convert"] + png_files + ["+append"] + [png_file_pooled]                
        call(command)    
        all_pngs.append(png_file_pooled)
        
    command = ["convert"] + all_pngs + ["-append"] + [os.path.join(model_dir, "synthetic_meshes/all_z.png")]
    call(command)

In [58]:
# experiments = ["2020-09-30_12-36-43", "2020-09-30_12-36-48", "2020-09-11_02-13-41"]
# experiments = ["2020-09-11_02-13-41"]
experiments = ["2020-09-30_12-36-48"]


In [59]:
for experiment in experiments:
  create_figures(experiment)

2021-02-16 02:52:36,471 [ERROR] Unsupported point attribute type: point_data for file: ./template/template.vtk


z0
z1
z2
z3
z4
z5
z6
z7
